<a href="https://colab.research.google.com/github/rauledop1/my-portfolio/blob/main/0001_dashboard_hotspot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#%pip install leafmap duckdb mapclassify
import duckdb
import leafmap
import pandas as pd
import os

#Create a temporal DB
con = duckdb.connect()

#Load spatial and http extensions
con.sql('''
install httpfs;
load httpfs;
INSTALL spatial;
LOAD spatial;
''')

#from FIRMS
firms_sa='https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_South_America_48h.csv'
firms_ca='https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_Central_America_48h.csv'

#administrative boundary from url shp. https://hgl.harvard.edu/catalog
ARG='https://stacks.stanford.edu/file/druid:yb205cg6773/data.zip'
CHL='https://stacks.stanford.edu/file/druid:cy135qp9262/data.zip'
PER='https://stacks.stanford.edu/file/druid:gv908jn2631/data.zip'
COL='https://stacks.stanford.edu/file/druid:mj028ht5751/data.zip'
BRA='https://stacks.stanford.edu/file/druid:bs788qq4876/data.zip'
MEX='https://stacks.stanford.edu/file/druid:xq016gn9637/data.zip'
PER='https://stacks.stanford.edu/file/druid:gv908jn2631/data.zip'

#Download to local all information csv and shp files
country = [ARG,CHL,MEX,BRA,PER,COL]
firms=[firms_sa,firms_ca]
for i in country:
  leafmap.download_file(i, unzip=True)
  os.remove('data.zip')
for i in firms:
  leafmap.download_file(i, overwrite=True)

#Create an union of all countries downloaded
con.sql('''
  create table paises as
  select * from st_read('CHL_adm2.shp')
  union
  select * from st_read('ARG_adm2.shp')
  union
  select * from st_read('BRA_adm2.shp')
  union
  select * from st_read('MEX_adm2.shp')
  union
  select * from st_read('COL_adm2.shp')
  union
  select * from st_read('PER_adm2.shp')
  ''')

#Create spatial intersection and save the result into csv file
con.sql('''
with fire as ((select
  case when daynight='N' then 'Night' else 'Day' end as daynight,
  satellite,
  (acq_date||' '||left(acq_time,2)||':'||right(acq_time,2)||':00')::datetime as datetime,
  st_point(longitude::numeric, latitude::numeric) as geom
  from st_read('MODIS_C6_1_South_America_48h.csv'))
  union
  (select
  case when daynight='N' then 'Night' else 'Day' end as daynight,
  satellite,
  (acq_date||' '||left(acq_time,2)||':'||right(acq_time,2)||':00')::datetime as datetime,
  st_point(longitude::numeric, latitude::numeric) as geom
  from st_read('MODIS_C6_1_Central_America_48h.csv'))
  )
select p.ISO,
p.NAME_0,
p.NAME_1,
p.NAME_2,
p.HASC_2,
f.daynight,
f.satellite,
f.datetime,
st_x(f.geom) as longitude,
st_y(f.geom) as latitude
from paises p
inner join fire f on ST_Intersects(p.geom, f.geom)
''').to_csv('/content/drive/MyDrive/Colab Notebooks/0001_Dashboard_portfolio/firms_48h.csv')

Downloading...
From: https://stacks.stanford.edu/file/druid:yb205cg6773/data.zip
To: /content/data.zip
100%|██████████| 2.20M/2.20M [00:01<00:00, 1.38MB/s]


Extracting files...


Downloading...
From: https://stacks.stanford.edu/file/druid:cy135qp9262/data.zip
To: /content/data.zip
100%|██████████| 7.22M/7.22M [00:04<00:00, 1.77MB/s]


Extracting files...


Downloading...
From: https://stacks.stanford.edu/file/druid:xq016gn9637/data.zip
To: /content/data.zip
100%|██████████| 5.16M/5.16M [00:02<00:00, 1.75MB/s]


...

Downloading...
From: https://stacks.stanford.edu/file/druid:gv908jn2631/data.zip
To: /content/data.zip
100%|██████████| 2.46M/2.46M [00:01<00:00, 1.67MB/s]


Extracting files...


Downloading...
From: https://stacks.stanford.edu/file/druid:mj028ht5751/data.zip
To: /content/data.zip
100%|██████████| 1.12M/1.12M [00:00<00:00, 1.46MB/s]


Extracting files...


Downloading...
From: https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_South_America_48h.csv
To: /content/MODIS_C6_1_South_America_48h.csv
100%|██████████| 2.20M/2.20M [00:00<00:00, 24.5MB/s]
Downloading...
From: https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_Central_America_48h.csv
To: /content/MODIS_C6_1_Central_America_48h.csv
100%|██████████| 59.6k/59.6k [00:00<00:00, 7.83MB/s]
